# **8) Graph query helpers**

In [1]:
from pathlib import Path
import os, sys, platform, json, subprocess

ROOT = Path.cwd().resolve().parents[0]

if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)
print("Python:", sys.version)
print("OS:", platform.platform())

Project root: D:\IIT BBS\Job Resources\fiolabs\diagram-intel
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
OS: Windows-10-10.0.26100-SP0


In [2]:
from src.config.loader import load_cfg
from src.graph.queries import load_graph, find_nodes_by_text, nets_table, shortest_path_between_labels, extract_subgraph, save_query_result
from pathlib import Path

cfg = load_cfg()
pdf_stem = Path(next(Path(cfg.paths.input_pdfs).glob("*.pdf"))).stem
G = load_graph(cfg, pdf_stem, page=1)

In [3]:
# 1) Biggest nets
nets = nets_table(cfg, pdf_stem, page=1)
len(nets), nets[:3]

(158,
 [{'net_id': 0,
   'nodes': 4160,
   'components': 4,
   'ports': 2117,
   'junctions': 2039,
   'phase': 'L1/L2/L3/N',
   'voltage': None},
  {'net_id': 42,
   'nodes': 7,
   'components': 0,
   'ports': 0,
   'junctions': 7,
   'phase': None,
   'voltage': None},
  {'net_id': 16,
   'nodes': 4,
   'components': 0,
   'ports': 0,
   'junctions': 4,
   'phase': None,
   'voltage': None}])

In [4]:
# 2) Find “Lift” nodes
lift_nodes = find_nodes_by_text(G, r"\blift\b", kinds=["component","port"])
lift_nodes[:3]

[('comp:wiring diagram_Main Distribution Panel_Fio Labs:1:comp:5:4:438942',
  {'kind': 'component',
   'comp_id': 'wiring diagram_Main Distribution Panel_Fio Labs:1:comp:5:4:438942',
   'bbox': [3480.0, 3480.0, 10524.0, 7442.0],
   'type': None,
   'confidence': 0.0,
   'labels_context': '1 3 | 125G | 151G | 152G | 154G | MCB4  32A | TPN2 63A | To | TPN3 63A | MCB5  32A | 155G | 157G | 170G | 20G | MCB9  32A | MCB10  32A | TPN5,6 63A | 202G | 204G | 205G | 212G | 214G | 217G | 220G | 221G | 142G | MCB6  32A | 55K | 55G | TPN1 100A | 215G | 57K | 70K | 20K | 57G | 70G | 172G | MCB7  25A | 174G | 175G | 150G | MCB8  25A | TPN4 63A | 177G | 200G | 201G | 122G | 124G | 121G | 120G | Lavis Engineering India Pvt. Ltd. | #12-13-832, Gokul Nagar, Tarnaka, Secunderabad-500017 | Telengana, India. Web: www.lavisindia.com | Project Code: 222106 | Size: A4 | Title: Internal Wiring Diagram of MDB -FIO LABS-222106 | Rev: 01 | 127G | 140G | 141G | 144G | 145G | 147G | From Stabilizer | first floor | S

In [5]:
# 3) Find a likely incomer (grid/dg) and the path to “Lift”
path1 = shortest_path_between_labels(G, r"(incoming|incomer|mains|grid|from\s*meter|eb)", r"\blift\b")
path1

{'path': ['comp:wiring diagram_Main Distribution Panel_Fio Labs:1:comp:5:4:438942'],
 'length': 1,
 'src': 'comp:wiring diagram_Main Distribution Panel_Fio Labs:1:comp:5:4:438942',
 'dst': 'comp:wiring diagram_Main Distribution Panel_Fio Labs:1:comp:5:4:438942'}

In [6]:
# 4) Save a small subgraph around the first “Lift” result for UI debugging
if lift_nodes:
    sg = extract_subgraph(G, lift_nodes[0][0], hops=2)
    # Serialize for inspection (node ids + edge list)
    data = {
        "nodes": [{"id": n, **a} for n,a in sg.nodes(data=True)],
        "edges": [{"u": u, "v": v, **attrs} for u,v,attrs in sg.edges(data=True)],
    }
    save_query_result(cfg, pdf_stem, "lift_subgraph_h2", data, page=1)